In [1]:
from carga_dados import CargaDados
from limpeza_dados import LimpezaDados

In [6]:
cg = CargaDados()
df_seg = cg.executar()
lp = LimpezaDados(df_seg)
df_seg = lp.executar()


INFO:root:Carregando dados de segmentos...
INFO:root:206488 registros carregados.
INFO:root:Etapa limpeza dados...
